# Generate Latent Space Map using trained encoder + classifier

## 0. Imports

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from tensorflow import keras
from matplotlib import pyplot as plt

## 1. Load SOTA classifier

In [ ]:
classifier = keras.models.load_model('training/saved_models/best_classifier.h5', compile=False)
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## 2. Load SOTA decoder (Latent Space Z -> Img)

In [ ]:
decoder = keras.models.load_model('training/saved_models/best_decoder.h5')

## 3. Generate grid of points

In [ ]:
x = np.linspace(0,1,500)
y = np.linspace(0,1,500)
grid = np.array([(a,b) for a,b in product(x,y)])

## 4. Decode the grid to generated images using the decoder

In [ ]:
decoder_preds = decoder.predict(grid)

# Save to reuse later
np.save('artifacts/decoder_preds.npy', decoder_preds)

print(decoder_preds.shape)

## 5. Classify the prediction of decoder into classes using classifier

In [ ]:
classifier_preds = classifier.predict(decoder_preds)

print(classifier_preds.shape)

classifier_preds = np.argmax(classifier_preds, axis=1)

# Save to reuse later (this is actually a useful artifact for mapSpace function)
np.save('artifacts/classifier_preds.npy', classifier_preds)

## 6. Build and display map

In [ ]:
# Define your color map using hexadecimal color codes
color_map = {
    0: 'red',
    1: 'blue',
    2: 'green',
    3: 'purple',
    4: 'orange',
    5: 'cyan',
    6: 'magenta',
    7: 'yellow',
    8: 'lime',
    9: 'brown',
}

# Create a mapping of class labels to text labels
class_labels = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot',
}

# Load classifier predictions (replace this with your actual data)
classifier_preds = np.load('artifacts/classifier_preds.npy')

# Create a DataFrame
df = pd.DataFrame(grid)
df.rename(columns={0:'x', 1:'y'}, inplace=True)
df['label'] = [class_labels[label] for label in classifier_preds]
df['y'] = 1 - df['y']  # Invert y values

# Create a scatter plot using Matplotlib
plt.figure(figsize=(6, 6))
for label, color in color_map.items():
    label_df = df[df['label'] == class_labels[label]]
    plt.scatter(label_df['x'], label_df['y'], color=color, label=class_labels[label], s=1.0, alpha=1.0, edgecolors='none')

# Customize the plot
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.xticks([])
plt.yticks([])

# Save the image without borders
plt.savefig("latentMap.png", dpi=300, bbox_inches='tight', pad_inches=0.0)
